In [1]:
import torch
import yaml
import os

# Humandoid MARL
from Humanoid_MARL.envs.base_env import GymWrapper, VectorGymWrapper
from Humanoid_MARL.utils.torch_utils import save_models, load_models, load_models_empty
from Humanoid_MARL.agent.ppo.train_torch import Agent, eval_unroll, get_agent_actions
from Humanoid_MARL.envs.torch_wrapper import TorchWrapper
from IPython.display import HTML, clear_output
from brax.io import html
import jax
from Humanoid_MARL import envs
from Humanoid_MARL.utils.utils import load_reward_config, load_config

In [2]:
env_name = "point_mass"
config = load_config(env_name)
env = envs.create(config['env_name'], auto_reset=True, **config['env_config'])
env = GymWrapper(env, get_jax_state=True)
env = TorchWrapper(env, device=config['train_config']['device'], get_jax_state=True)

model_path = None
observation = env.reset()
action = torch.zeros(env.action_space.shape[0] * env.num_agents).to(config['train_config']['device'])
env.step(action)
agents = load_models_empty(model_path, Agent, device=config['train_config']['device'], network_config=config['network_config'], training_config=config['train_config'], num_agents=1) 

In [3]:
jax_states = []
num_steps = 10
eval_reward = 0.0
episodes = torch.zeros((), device = config['train_config']['device'])
for i in range(num_steps):
    print(f"{i} / {num_steps}")
    action = torch.ones(env.action_space.shape[0] * env.num_agents).to(config['train_config']['device'])
    jax_state, observation, reward, done, info = env.step(Agent.dist_postprocess(action))
    episodes += torch.sum(done)
    jax_states.append(jax_state)
    eval_reward += reward
    if done:
        observation = env.reset()
print(f"Total Reward | {eval_reward / episodes}")

0 / 10
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
Total Reward | tensor([nan], device='cuda:0')


In [4]:
HTML(html.render(env.sys, [jax_state.pipeline_state for jax_state in jax_states])) 

In [5]:
HTML(html.render(env.sys, [jax_state.pipeline_state for jax_state in jax_states])) 